# Linear regression implementations

In [13]:
# set up working catalog
import sys
from pathlib import Path
sys.path.append(str(Path("..")))

# imports
from common.utils import get_numeric_data, split

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
 

In [14]:
data = get_numeric_data()
data


,Previous qualification (grade),Admission grade,Age at enrollment,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,122.0,127.3,20,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,10.8,1.4,1.74
1,160.0,142.5,19,0,6,6,6,14.000000,0,0,6,6,6,13.666667,0,13.9,-0.3,0.79
2,122.0,124.8,19,0,6,0,0,0.000000,0,0,6,0,0,0.000000,0,10.8,1.4,1.74
3,122.0,119.6,20,0,6,8,6,13.428571,0,0,6,10,5,12.400000,0,9.4,-0.8,-3.12
4,100.0,141.5,45,0,6,9,5,12.333333,0,0,6,6,6,13.000000,0,13.9,-0.3,0.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,125.0,122.2,19,0,6,7,5,13.600000,0,0,6,8,5,12.666667,0,15.5,2.8,-4.06
4420,120.0,119.0,18,0,6,6,6,12.000000,0,0,6,6,2,11.000000,0,11.1,0.6,2.02
4421,154.0,149.5,30,0,7,8,7,14.912500,0,0,8,9,1,13.500000,0,13.9,-0.3,0.79
4422,180.0,153.8,20,0,5,5,5,13.800000,0,0,5,6,5,12.000000,0,9.4,-0.8,-3.12


Separate data into training and test sets

In [15]:
X = data.drop("Admission grade", axis=1)
X = StandardScaler().fit_transform(X)       # normalize
y = data["Admission grade"]

X_train, X_val, X_test, y_train, y_val, y_test = split(X, y)

In [16]:
class CustomLinearRegression:
    def __init__(self):
        self.coefficients = None
        
    def fit_base(self, X, y):
        if isinstance(X, pd.DataFrame):
            X_matrix = np.c_[np.ones(shape=(X.shape[0],1)), X.values]
        else: 
            X_matrix = np.c_[np.ones(shape=(X.shape[0],1)), X]
            
        if isinstance(y, pd.Series):
            y_matrix = y.values.reshape(-1, 1)
        else: 
            y_matrix = y.reshape(-1, 1)
            
        return X_matrix, y_matrix
        
    def predict(self, X):
        if self.coefficients is None:
            raise ValueError("Model is not trained.")
        
        if isinstance(X, pd.DataFrame):
            return np.c_[np.ones((X.shape[0], 1)), X.values] @ self.coefficients
        
        return np.c_[np.ones((X.shape[0], 1)), X] @ self.coefficients

My implementation of the closed form solution for linear regression

  $y = Xw$

  $w = (X^T X)^{-1} X^T y$

In [17]:
class ClosedFormSolution(CustomLinearRegression):
    def __init__(self):
        super().__init__()
    
    def fit(self, X, y):
        X_matrix, y_matrix = super().fit_base(X, y)
        self.coefficients = np.linalg.inv(X_matrix.T @ X_matrix) @ X_matrix.T @ y_matrix

My implementation of the gradient descent solution for linear regression

  $y = Xw$

  $w := w - \alpha \cdot \nabla J(w)$

  $\nabla J(w) = \frac{2}{m} X^T (Xw - y)$

(cost function)

In [18]:
class GradientDescent(CustomLinearRegression):
    def __init__(self):
        super().__init__()
        
    def fit(self, X, y, **kwargs):
        iterations = kwargs.get('iterations', 500)
        batch_size = kwargs.get('batch_size', None)
        learning_rate = kwargs.get('learning_rate', 0.002)
        
        X_matrix, y_matrix = super().fit_base(X, y)
        
        samples_count, coef_count = X_matrix.shape
        
        self.coefficients = np.zeros((X_matrix.shape[1], 1))
        
        for _ in range(iterations):
            if batch_size:
                indices = np.random.choice(samples_count, batch_size, replace = False)
                X_batch = X_matrix[indices]
                y_batch = y_matrix[indices]
            else:
                X_batch = X_matrix
                y_batch = y_matrix
                
            factor = (2 / (batch_size if batch_size else samples_count)) 
            gradients = factor * X_batch.T @ (X_batch @ self.coefficients - y_batch)
                    
            self.coefficients -= learning_rate * gradients
            

In [19]:
stdout_sep = "\t"

def print_result(model, **kwargs):
    model.fit(X_train, y_train, **kwargs)
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    y_pred_test = model.predict(X_test)
    
    print(
        f"\tTrain:\t\tMSE: {mean_squared_error(y_train, y_pred_train):.4f}",
        stdout_sep,
        f"R2: {r2_score(y_train, y_pred_train):.4f}"
    )
    print(
        f"\tValidation:\tMSE: {mean_squared_error(y_val, y_pred_val):.4f}",
        stdout_sep,
        f"R2: {r2_score(y_val, y_pred_val):.4f}"
    )
    print(
        f"\tTrain:\t\tMSE: {mean_squared_error(y_test, y_pred_test):.4f}",
        stdout_sep,
        f"R2: {r2_score(y_test, y_pred_test):.4f}"
    )
    

Closed form solution linear regression

In [20]:
PARAM_CANDIDATES = {
    "iterations": [500, 1000, 2500, 5000, 7500],
    "batch_sizes": [None, 64, 128, 256],
    "learning_rates": [0.0005, 0.001, 0.005]
}

def find_gradient_best_params(X_train, y_train, X_val, y_val):
    
    model = GradientDescent()
    best_mse = float('inf')
    best_params = None
    
    for iterations in PARAM_CANDIDATES["iterations"]:
        for batch_size in PARAM_CANDIDATES["batch_sizes"]:
            for learning_rate in PARAM_CANDIDATES["learning_rates"]:
                kwargs = {
                    "iterations": iterations, 
                    "batch_size": batch_size, 
                    "learning_rate": learning_rate
                }
                model.fit(X_train, y_train, **kwargs)
                mse = mean_squared_error(y_val, model.predict(X_val))
                if mse < best_mse:
                    best_mse = mse
                    best_params = kwargs.copy()
                
        # log (but it's not log XD)    
        print("\titerations = ", iterations)
                    
    return best_params
    

In [21]:
print("===== FINDING BEST PARAMS =====")
best_params = find_gradient_best_params(X_train, y_train, X_val, y_val)
print(f"Best params: {best_params}")
print("\n===== GRADIENT DESCENT LINEAR REGRESSION =====")
print_result(GradientDescent(), **best_params)

===== FINDING BEST PARAMS =====
	iterations =  500
	iterations =  1000
	iterations =  2500
	iterations =  5000
	iterations =  7500
Best params: {'iterations': 2500, 'batch_size': 128, 'learning_rate': 0.005}

===== GRADIENT DESCENT LINEAR REGRESSION =====
	Train:		MSE: 131.1540 	 R2: 0.3589
	Validation:	MSE: 125.2117 	 R2: 0.4395
	Train:		MSE: 153.4285 	 R2: 0.3029


In [22]:
print("===== CLOSED FORM SOLUTION LINEAR REGRESSION =====")
print_result(ClosedFormSolution())

===== CLOSED FORM SOLUTION LINEAR REGRESSION =====
	Train:		MSE: 130.9588 	 R2: 0.3598
	Validation:	MSE: 124.2138 	 R2: 0.4440
	Train:		MSE: 152.8460 	 R2: 0.3056


In [23]:
print("\n===== SKLEARN LINEAR REGRESSION =====")
print_result(LinearRegression())


===== SKLEARN LINEAR REGRESSION =====
	Train:		MSE: 130.9588 	 R2: 0.3598
	Validation:	MSE: 124.2138 	 R2: 0.4440
	Train:		MSE: 152.8460 	 R2: 0.3056


# Conclusions
1) The results are basically the same for sklearn and closed form solution implementation
2) Gradient descent gives nearly the same results